In [1]:
import torch
import torch.nn as nn
from torch.utils.data import IterableDataset, DataLoader
import torch.nn.functional as F

from random import random, randint

from typing import Dict, Tuple, List
from torch import Tensor

import pickle
import os
import glob

from torchtext.vocab import Vocab

from griffon.utils import pad_list

import numpy as np

from griffon.coq_dataclasses import *
from griffon.constants import *
from griffon.dataset.count_dataset import CounTDataset

import random

random.seed(0)

In [2]:
ds = CounTDataset("../data/CounT/train", "../models/vocab.pickle")
dl = DataLoader(ds, 10000, collate_fn=ds.collate_fn)


AssertionError: Path ../data/CounT/train does not exist

In [8]:
print(ds.vocab.get_itos()[:8])

test = pickle.load(open("../data/small/stage2/train/proof00000002.pickle", "rb"))
print(test.hypotheses[1].tokens)

test_2 = next(iter(ds))
print(test_2.input_ids)

['<sos>', '<unk>', '<mask>', '<pad>', 'Coq', 'Init', '.', '(']
[Stage2Token(subtokens=[1], original_subtokens=['IHn']), Stage2Token(subtokens=[12], original_subtokens=['SerTop']), Stage2Token(subtokens=[52], original_subtokens=['ev']), Stage2Token(subtokens=[12], original_subtokens=['SerTop']), Stage2Token(subtokens=[51], original_subtokens=['double']), Stage2Token(subtokens=[11], original_subtokens=['n'])]
tensor([[ 161,    3,    3,    3,    3],
        [  12,    3,    3,    3,    3],
        [2346,    3,    3,    3,    3],
        [  77,    3,    3,    3,    3]])


In [3]:
random.seed(0)
pad_idx = ds.vocab[PAD_TOKEN]
print(pad_idx)

def print_sample(vocab:Vocab, sample:CounTSample):
    itos = ds.vocab.get_itos()
    
    def pretty_print(ids:Tensor):
        tokens = ids.tolist()
        print(" ".join(["_".join([itos[subtoken] for subtoken in token if subtoken != pad_idx]) for token in tokens]))

    print("sample")
#    print("super original token")
#    print(sample.input_ids.tolist())
    original_tokens = torch.clone(sample.input_ids)
    if torch.count_nonzero(sample.target_mask):
        original_tokens[sample.target_mask] = sample.target_ids

    print("original")
    pretty_print(original_tokens)
    print("input")
    pretty_print(sample.input_ids)
    print("-----------------")

for i in range(10):
    print(i)
    print_sample(ds.vocab, ds[i])
    if i == 10:
        break

3
0
sample
original
H0 Logic_Init_Coq eq Registers_backend_compcert reg SerTop ereg y SerTop ereg z
input
H0 Logic_Init_Coq eq <mask>_<mask>_<mask>_<mask>_<mask> reg SerTop ereg y SerTop ereg z
-----------------
1
sample
original
H1 Logic_Init_Coq or Locations_backend_compcert OrderedLoc lt SerTop eloc y SerTop eloc z Logic_Init_Coq and Logic_Init_Coq eq Locations_backend_compcert loc SerTop eloc y SerTop eloc z SerTop OrderedEqKind lt SerTop ekind y SerTop ekind z
input
H1 Logic_Init_Coq <mask>_<mask>_<mask>_<mask>_<mask> Locations_backend_compcert OrderedLoc lt SerTop <mask>_<mask>_<mask>_<mask>_<mask> y SerTop eloc z Logic_Init_Coq and Logic_Init_Coq eq Locations_backend_compcert loc SerTop <mask>_<mask>_<mask>_<mask>_<mask> y SerTop eloc <mask>_<mask>_<mask>_<mask>_<mask> SerTop OrderedEqKind lt SerTop ekind y SerTop ekind z
-----------------
2
sample
original
goal Logic_Init_Coq or Coqlib_lib_compcert Plt SerTop ereg x SerTop ereg z Logic_Init_Coq and Logic_Init_Coq eq Registers_b

In [5]:
from torch.utils.data.dataloader import DataLoader

def print_batch(vocab:Vocab, inp:CounTInput, tgt:CounTTarget):
    itos = ds.vocab.get_itos()

    def pretty_print(ids:Tensor):
        tokens = ids.tolist()
        print(" ".join(["_".join([itos[subtoken] for subtoken in token if subtoken != pad_idx]) for token in tokens]))

    for b in range(inp.input_ids.shape[0]):
        print("sample")
        original_tokens = torch.clone(inp.input_ids[b][[inp.input_padding_mask[b]]])
        if torch.count_nonzero(tgt.target_mask[b][tgt.target_padding_mask[b]]):
            original_tokens[tgt.target_mask[b][tgt.target_padding_mask[b]]] = tgt.target_ids[b][tgt.target_padding_mask[b]]

        print("original")
        pretty_print(original_tokens)
        print("input")
        pretty_print(inp.input_ids[b][[inp.input_padding_mask[b]]])
        print("-----------------")
        
dl = DataLoader(ds, 10, collate_fn=ds.collate_fn)
for inp, tgt in dl:
    print(type(inp))
    print_batch(ds.vocab, inp, tgt)
    break

<class 'griffon.coq_dataclasses.CounTInput'>
sample


IndexError: The shape of the mask [12] at index 0 does not match the shape of the indexed tensor [55] at index 0